# Machine Learning Project
# Kansas City Crime Data Deep Dive

### Import Dependancies

In [1]:
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas import datetime
from dateutil import parser
from scipy import stats
import glob
import os

### Combine KCMO crime data (Raw data came from data.kcmo.org)

In [2]:
path = r'C:\DataAnalytics\003_Project_Three\Resources'
all_files = glob.glob(os.path.join(path, '*.csv'))

next_df = (pd.read_csv(f) for f in all_files)

combined = pd.concat(next_df, ignore_index=True)
combined.head()

,Address,Age,Area,Beat,City,DVFlag,Description,Firearm Used Flag,Firearm Used Flag,From Time,...,Rep_Dist,Report_No,Reported Time,Reported_Date,Reported_Time,Sex,To Time,To_Date,To_Time,Zip Code
0,PROSPECT AV/VICTOR ST,28.0,EPD,333,KANSAS CITY,U,Possession of Drug E,NaN,N,22:56,...,PJ2327,70059279,3:24,10/06/2009,NaN,F,NaN,10/05/2009 12:00:00 AM,23:10,64128.0
1,1100 LOCUST ST,27.0,CPD,112,KANSAS CITY,U,Justifiable Homicide,NaN,Y,12:00,...,PJ1029,80005443,11:45,02/05/2009,NaN,M,NaN,NaN,NaN,64106.0
2,100 WHEELING AV,22.0,EPD,315,KANSAS CITY,U,Possession/Sale/Dist,NaN,N,21:15,...,PJ0371,80019629,22:50,06/18/2009,NaN,M,NaN,NaN,NaN,64129.0
3,E 44 ST/INDIANA AV,NaN,EPD,334,KANSAS CITY,U,Stolen Property OFFE,NaN,N,18:44,...,PJ3525,70060962,18:44,01/28/2009,NaN,U,NaN,NaN,NaN,64130.0
4,1100 LOCUST ST,28.0,CPD,112,KANSAS CITY,U,Justifiable Homicide,NaN,Y,12:00,...,PJ1029,80005443,11:45,02/05/2009,NaN,M,NaN,NaN,NaN,64106.0


### Features in dataset

* Age

### Change time to datetime and extract hour

In [3]:
# Placed below to be run on dataframe when its smaller

# combined.Reported_Date = pd.to_datetime(combined.Reported_Date)
# combined.Reported_Time = pd.to_datetime(combined.Reported_Time)
# combined.Reported_Time = combined.Reported_Time.dt.floor('h')
# combined.Reported_Time = combined.Reported_Time.dt.hour

### Rename & drop columns

In [7]:
combined.rename(columns={'Zip Code':'zip_code'}, inplace=True)
combined_dropped_columns = combined.drop(['Report_No', 'To_Date', 'From_Date', 'To_Time', 'From_Time',
                          'Offense', 'IBRS', 'Rep_Dist', 'Area', 'Beat', 'DVFlag', 'Address'], axis=1)

### Filter for male and female victims age 90 and under in KCMO

In [8]:
kc_crime_clean_zips = combined_dropped_columns[combined_dropped_columns.zip_code != 99999]
only_kc_crime = kc_crime_clean_zips[kc_crime_clean_zips.City.str.contains("KANSAS CITY") == True]

victims = only_kc_crime[only_kc_crime.Involvement.str.contains("VIC") == True]

victims_no_nans = victims[victims.Sex.str.contains("NaN") == False]
male_female_victims_kcmo = victims_no_nans[victims_no_nans.Sex.str.contains("U") == False]

kc_crime_real_ages = male_female_victims_kcmo[male_female_victims_kcmo['Age'] < 91]

In [12]:
kc_crime_real_ages.to_csv('check_for_duplicate_columns_V2.csv', index=False)

In [14]:
from_excel = os.path.join('check_for_duplicate_columns_V2.csv')
combined_and_partially_cleaned_in_excel = pd.read_csv(from_excel)
combined_and_partially_cleaned_in_excel.head()

,Age,City,Description,Firearm Used Flag,From Time,Invl_No,Involvement,Location,Race,Reported Time,Reported_Date,Reported_Time,Sex,To Time,zip_code
0,24.0,KANSAS CITY,Burglary_Residence,N,NaN,1.0,VIC,100 ASKEW AV\r\nKANSAS CITY 64126\r\n(39.11296...,B,NaN,12/6/2017,15:17,F,NaN,64126.0
1,52.0,KANSAS CITY,Agg_Assault_Domest,N,NaN,1.0,VIC,100 BELLAIRE AV\r\nKANSAS CITY 64123\r\n(39.11...,W,NaN,2/5/2017,19:10,F,NaN,64123.0
2,24.0,KANSAS CITY,Agg_Assault_Domest,N,NaN,1.0,VIC,100 BELLAIRE AV\r\nKANSAS CITY 64123\r\n(39.11...,W,NaN,4/9/2017,15:09,F,NaN,64123.0
3,59.0,KANSAS CITY,Non_Agg_Assault_Dome,N,NaN,1.0,VIC,100 BELLAIRE AV\r\nKANSAS CITY 64123\r\n(39.11...,W,NaN,4/13/2017,8:54,F,NaN,64123.0
4,33.0,KANSAS CITY,Violation_Full_Order,N,NaN,1.0,VIC,100 BELLAIRE AV\r\nKANSAS CITY 64123\r\n(39.11...,A,NaN,5/16/2017,21:06,F,NaN,64123.0


### Copy kc_crime_real_ages and separate "Location" into 3 columns

In [15]:
combined_and_partially_cleaned_in_excel_copy = combined_and_partially_cleaned_in_excel.copy()
combined_and_partially_cleaned_in_excel_copy.head(1)

,Age,City,Description,Firearm Used Flag,From Time,Invl_No,Involvement,Location,Race,Reported Time,Reported_Date,Reported_Time,Sex,To Time,zip_code
0,24.0,KANSAS CITY,Burglary_Residence,N,NaN,1.0,VIC,100 ASKEW AV\r\nKANSAS CITY 64126\r\n(39.11296...,B,NaN,12/6/2017,15:17,F,NaN,64126.0


# Split out geo data

In [16]:
location_only = combined_and_partially_cleaned_in_excel_copy['Location'].str[0:-1].str.split('\n', expand=True)
location_only.columns = ("address", "city_zip", "geo")
location_only.head(100)

,address,city_zip,geo
0,100 ASKEW AV\r,KANSAS CITY 64126\r,"(39.112962, -94.538718"
1,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
2,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
3,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
4,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
5,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
6,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
7,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
8,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"
9,100 BELLAIRE AV\r,KANSAS CITY 64123\r,"(39.112246, -94.51289"


### Parsing out latitude and longitude

In [17]:
location_only['geo'] = location_only['geo'].str[1:]
geo_split = location_only['geo'].str[0:].str.split(', ', expand=True)
geo_split.columns = ("Latitude", "Longitude")
geo_split.head(1)

,Latitude,Longitude
0,39.112962,-94.538718


In [18]:
kc_crime_real_ages_copy.count()

Age                    355200
City                   355200
Description            355200
Firearm Used Flag      123738
Firearm Used Flag      231462
From Time              270346
Invl_No                355200
Involvement            355200
Latitude                80708
Location                43030
Location 1             311903
Longitude               80708
Race                   355200
Reported Time          270852
Reported_Date          355200
Reported_Time           84348
Sex                    355200
To Time                109590
zip_code               348842
dtype: int64

In [19]:
geo_split.count()

Latitude     337570
Longitude    337570
dtype: int64

In [20]:
kcmo_crime_with_nans = pd.concat([combined_and_partially_cleaned_in_excel_copy, geo_split], axis=1)
kcmo_crime_with_nans.head(1)

,Age,City,Description,Firearm Used Flag,From Time,Invl_No,Involvement,Location,Race,Reported Time,Reported_Date,Reported_Time,Sex,To Time,zip_code,Latitude,Longitude
0,24.0,KANSAS CITY,Burglary_Residence,N,NaN,1.0,VIC,100 ASKEW AV\r\nKANSAS CITY 64126\r\n(39.11296...,B,NaN,12/6/2017,15:17,F,NaN,64126.0,39.112962,-94.538718


In [21]:
kcmo_crime_with_nans.count()

Age                  348920
City                 348920
Description          348920
Firearm Used Flag    348920
From Time            270346
Invl_No              348920
Involvement          348920
Location             348653
Race                 348920
Reported Time        270852
Reported_Date        348920
Reported_Time         78068
Sex                  348920
To Time              109590
zip_code             342562
Latitude             337570
Longitude            337570
dtype: int64

In [22]:
kcmo_crime_no_lat_nans = kcmo_crime_with_nans[kcmo_crime_with_nans.Latitude.str.contains("NaN") == False]
kcmo_crime_no_nans = kcmo_crime_no_lat_nans[kcmo_crime_no_lat_nans.Longitude.str.contains("NaN") == False]
kc_crime_for_visualizations = kcmo_crime_no_nans.drop(['City', 'Invl_No', 'Involvement', 'Location'], axis=1)
kc_crime_for_visualizations.head(1)

,Age,Description,Firearm Used Flag,From Time,Race,Reported Time,Reported_Date,Reported_Time,Sex,To Time,zip_code,Latitude,Longitude
0,24.0,Burglary_Residence,N,NaN,B,NaN,12/6/2017,15:17,F,NaN,64126.0,39.112962,-94.538718


### Change time to datetime and extract hour

In [23]:
kc_crime_for_visualizations.Reported_Date = pd.to_datetime(kc_crime_for_visualizations.Reported_Date)
kc_crime_for_visualizations.Reported_Time = pd.to_datetime(kc_crime_for_visualizations.Reported_Time)
kc_crime_for_visualizations.Reported_Time = kc_crime_for_visualizations.Reported_Time.dt.floor('h')
kc_crime_for_visualizations.Reported_Time = kc_crime_for_visualizations.Reported_Time.dt.hour

In [25]:
kc_crime_for_visualizations.columns = ( "Age", "Crime", "Firearm", "Needs Dropped", "Race", "Needs Combined", "Date", "Time", "Sex", "Needs Dropped", "Zip", "Latitude", "Longitude")
kc_crime_for_visualizations = kc_crime_for_visualizations.reset_index(drop=True)
kc_crime_for_visualizations.head(100)

,Age,Crime,Firearm,Needs Dropped,Race,Needs Combined,Date,Time,Sex,Needs Dropped,Zip,Latitude,Longitude
0,24.0,Burglary_Residence,N,NaN,B,NaN,2017-12-06,15.0,F,NaN,64126.0,39.112962,-94.538718
1,52.0,Agg_Assault_Domest,N,NaN,W,NaN,2017-02-05,19.0,F,NaN,64123.0,39.112246,-94.51289
2,24.0,Agg_Assault_Domest,N,NaN,W,NaN,2017-04-09,15.0,F,NaN,64123.0,39.112246,-94.51289
3,59.0,Non_Agg_Assault_Dome,N,NaN,W,NaN,2017-04-13,8.0,F,NaN,64123.0,39.112246,-94.51289
4,33.0,Violation_Full_Order,N,NaN,A,NaN,2017-05-16,21.0,F,NaN,64123.0,39.112246,-94.51289
5,33.0,Non_Agg_Assault_Dome,N,NaN,A,NaN,2017-05-16,21.0,F,NaN,64123.0,39.112246,-94.51289
6,83.0,Fraud_Confidence_Gamb,N,NaN,W,NaN,2017-08-26,9.0,M,NaN,64123.0,39.112246,-94.51289
7,32.0,Stolen_Property,N,NaN,W,NaN,2017-11-16,20.0,M,NaN,64123.0,39.112246,-94.51289
8,32.0,Auto_Theft_Outside,N,NaN,W,NaN,2017-11-16,20.0,M,NaN,64123.0,39.112246,-94.51289
9,49.0,Auto_Theft,N,NaN,U,NaN,2017-11-25,18.0,M,NaN,64123.0,39.112246,-94.51289


# Do not proceed until ensuring clean data from above!

In [ ]:
kc_crime_for_visualizations.shape

In [ ]:
kc_crime_for_visualizations.count()

In [ ]:
kc_crime_for_visualizations.dtypes

### Confirming Clean Data in Excel

In [25]:
kc_crime_for_visualizations.to_csv('kc_crime_for_visualizations.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(for_split, test_size=0.2)

In [26]:
kc_crime_for_visualizations_copy = kc_crime_for_visualizations.copy()

# Confirm above data is good before proceeding

### Apply get_dummies function

In [29]:
crime_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Crime'],prefix = "Crime")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, crime_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Crime', axis=1, inplace=True)
zip_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Zip'],prefix = "Zip")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, zip_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Zip', axis=1, inplace=True)
race_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Race'],prefix = "Race")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, race_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Race', axis=1, inplace=True)
sex_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Sex'],prefix = "Sex")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, sex_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Sex', axis=1, inplace=True)
age_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Age'],prefix = "Age")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, age_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Age', axis=1, inplace=True)
date_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Date'],prefix = "Date")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, date_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Date', axis=1, inplace=True)
hour_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Hour'],prefix = "Hour")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, hour_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Hour', axis=1, inplace=True)
firearm_columns = pd.get_dummies(kc_crime_for_visualizations_copy['Firearm'],prefix = "Firearm")
kc_crime_for_visualizations_copy = pd.concat([kc_crime_for_visualizations_copy, firearm_columns], axis=1)
kc_crime_for_visualizations_copy.drop('Firearm', axis=1, inplace=True)

In [30]:
kc_crime_for_visualizations_copy.head()

,Latitude,Longitude,Crime_Agg_Assault_Domest,Crime_Agg_Assault_Drive,Crime_Aggravated_Assault,Crime_Armed_Robbery,Crime_Arson,Crime_Attempted_Suicide,Crime_Auto_Theft,Crime_Auto_Theft_Outside,...,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Firearm_N,Firearm_Y
0,39.053635,-94.595998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,39.10091,-94.577328,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,39.17744,-94.572069,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,39.033505,-94.547812,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,39.235881,-94.466171,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [31]:
kc_crime_for_visualizations_copy.to_csv('kc_crime_get_dummies.csv', index=False)

In [32]:
train, test = train_test_split(kc_crime_for_visualizations_copy, test_size=0.2)

In [33]:
test.head()

,Latitude,Longitude,Crime_Agg_Assault_Domest,Crime_Agg_Assault_Drive,Crime_Aggravated_Assault,Crime_Armed_Robbery,Crime_Arson,Crime_Attempted_Suicide,Crime_Auto_Theft,Crime_Auto_Theft_Outside,...,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Firearm_N,Firearm_Y
16357,38.950993,-94.487819,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
13207,39.166088,-94.484629,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6645,39.235282,-94.592211,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
24780,38.930723,-94.511843,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
15885,39.112289,-94.514051,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [34]:
test = test.reset_index(inplace = False)
test = test.drop(['index'], axis=1)

In [35]:
test.head()

,Latitude,Longitude,Crime_Agg_Assault_Domest,Crime_Agg_Assault_Drive,Crime_Aggravated_Assault,Crime_Armed_Robbery,Crime_Arson,Crime_Attempted_Suicide,Crime_Auto_Theft,Crime_Auto_Theft_Outside,...,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Firearm_N,Firearm_Y
0,38.950993,-94.487819,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,39.166088,-94.484629,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,39.235282,-94.592211,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,38.930723,-94.511843,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,39.112289,-94.514051,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [36]:
train.head()

,Latitude,Longitude,Crime_Agg_Assault_Domest,Crime_Agg_Assault_Drive,Crime_Aggravated_Assault,Crime_Armed_Robbery,Crime_Arson,Crime_Attempted_Suicide,Crime_Auto_Theft,Crime_Auto_Theft_Outside,...,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Firearm_N,Firearm_Y
28902,39.242051,-94.461697,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13631,39.10311,-94.544228,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
20832,39.107595,-94.535253,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3452,39.091129,-94.561035,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
8093,38.921127,-94.549654,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [37]:
train = train.reset_index(inplace = False)
train = train.drop(['index'], axis=1)

In [38]:
train.head()

,Latitude,Longitude,Crime_Agg_Assault_Domest,Crime_Agg_Assault_Drive,Crime_Aggravated_Assault,Crime_Armed_Robbery,Crime_Arson,Crime_Attempted_Suicide,Crime_Auto_Theft,Crime_Auto_Theft_Outside,...,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Firearm_N,Firearm_Y
0,39.242051,-94.461697,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,39.10311,-94.544228,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,39.107595,-94.535253,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,39.091129,-94.561035,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,38.921127,-94.549654,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
